In [11]:
import numpy as np
from docplex.mp.model import Model
import pandas as pd
from time import perf_counter

In [12]:
def solve_with_cplex(A, b, c):
    m, n = A.shape

    model = Model(name="Knapsack Problem")

    x = model.binary_var_list(n, name="x")

    for i in range(m):
        model.add_constraint(model.sum(A[i][j] * x[j] for j in range(n)) <= b[i])

    model.maximize(model.sum(c[j] * x[j] for j in range(n)))

    solution = model.solve()

    if solution:
        optimal_fitness = int(solution.objective_value)
        optimal_solution = [int(solution.get_value(x[j])) for j in range(n)]
        return optimal_fitness, optimal_solution
    else:
        return None, None

In [13]:
def test(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    A = []
    for i, line in enumerate(lines):
        if i == 0:
            n, m, optimal_fitness = [int(x) for x in line.split()]
        elif i == 1:
            c = [int(x) for x in line.split()]
        elif i > 1 and i <= m+1:
            A.append([int(x) for x in line.split()])
        else:
            b = [int(x) for x in line.split()]

    return A, b, c, optimal_fitness

In [16]:
cplex_df = pd.DataFrame()

for i in range(1, 21):
    A, b, c, optimal_fitness = test(file_path=f"../test_instances/test{i}.txt")
    print(f"Test {i}:")
    start = perf_counter()
    cplex_fitness, cplex_solution = solve_with_cplex(np.array(A), b, c)
    end = perf_counter()

    if cplex_fitness == optimal_fitness:
        cplex_df.loc[0, f'cplex managed to find an optimal solution for test {i}'] = 1
    else:
        cplex_df.loc[0, f'cplex managed to find an optimal solution for test {i}'] = 0

    cplex_df.loc[0, f'test {i} time'] = end - start
    
    print("CPLEX fitness:", cplex_fitness)
    print("And it should be:", optimal_fitness)
    print("CPLEX solution:", cplex_solution)
    print("-" * 30)

Test 1:
CPLEX fitness: 3800
And it should be: 3800
CPLEX solution: [0, 1, 1, 0, 0, 1]
------------------------------
Test 2:
CPLEX fitness: 4015
And it should be: 4015
CPLEX solution: [1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1]
------------------------------
Test 3:
CPLEX fitness: 6120
And it should be: 6120
CPLEX solution: [1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]
------------------------------
Test 4:
CPLEX fitness: 12400
And it should be: 12400
CPLEX solution: [1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]
------------------------------
Test 5:
CPLEX fitness: 10618
And it should be: 10618
CPLEX solution: [1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1]
------------------------------
Test 6:
CPLEX fitness: 16537
And it should be: 16537
CPLEX solution: [0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,

In [17]:
cplex_df

,cplex managed to find an optimal solution for test 1,test 1 time,cplex managed to find an optimal solution for test 2,test 2 time,cplex managed to find an optimal solution for test 3,test 3 time,cplex managed to find an optimal solution for test 4,test 4 time,cplex managed to find an optimal solution for test 5,test 5 time,...,cplex managed to find an optimal solution for test 16,test 16 time,cplex managed to find an optimal solution for test 17,test 17 time,cplex managed to find an optimal solution for test 18,test 18 time,cplex managed to find an optimal solution for test 19,test 19 time,cplex managed to find an optimal solution for test 20,test 20 time
0,1.0,0.030499,1.0,0.037096,1.0,0.029529,1.0,0.053623,1.0,0.028318,...,1.0,0.040603,1.0,0.040828,1.0,0.015142,1.0,0.044928,1.0,0.050183


In [18]:
cplex_df.to_csv('cplex.csv')